## 1. Import Dependencies

In [31]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
import re

## 2. Read post data into list

In [32]:
data = pd.read_csv('influencers_data.csv', encoding = "ISO-8859-1")

def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    #text = text.replace('\%','')
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    #text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('...see more', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = " ".join(filter(lambda x:x[0]!="@", text.split()))
    return text

corpus = data['content'].apply(lambda x: clean_text(x)).tolist()

## 3. Tokenization

In [33]:
tokenizer = Tokenizer(
    filters='"$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n'
)

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

## 4. Turn tokenized sentences into training data

In [34]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad Input Sequences
max_sequence_length = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre'))

# Create predictors and labels
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

## 5. Train neural network

In [35]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
history = model.fit(xs, ys, epochs=100, verbose=1)
#print model.summary()
print(model)

Epoch 1/100


/home/wingp/.local/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


83/83 [==============================] - 34s 384ms/step - loss: 7.5950 - accuracy: 0.0212
Epoch 2/100
83/83 [==============================] - 38s 463ms/step - loss: 7.7285 - accuracy: 0.0208
Epoch 3/100
83/83 [==============================] - 33s 398ms/step - loss: 7.2442 - accuracy: 0.0268
Epoch 4/100
83/83 [==============================] - 33s 403ms/step - loss: 6.5308 - accuracy: 0.0382
Epoch 5/100
83/83 [==============================] - 33s 397ms/step - loss: 6.1416 - accuracy: 0.0472
Epoch 6/100
83/83 [==============================] - 33s 403ms/step - loss: 5.8132 - accuracy: 0.0548
Epoch 7/100
83/83 [==============================] - 33s 395ms/step - loss: 5.4375 - accuracy: 0.0650
Epoch 8/100
83/83 [==============================] - 33s 398ms/step - loss: 5.0229 - accuracy: 0.0892
Epoch 9/100
83/83 [==============================] - 33s 400ms/step - loss: 4.5972 - accuracy: 0.1153
Epoch 10/100
83/83 [==============================] - 30s 358ms/step - loss: 4.1428 - accuracy

In [39]:
seed_text = "I think it is time to"
next_words = 200
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
    predicted = np.argmax(model.predict(token_list),axis=1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)

I think it is time to hire you had sex our hr asked a very attractive female candidate during an interview.she applied for an entry level role a few years back. sir i am married with two beautiful daughters. since we migrated here i've been unemployed for three years. i want to work and support my husband who's also out of work. we need financial help. i heard the noise from my office and quickly abandoned a meeting with my partner.we dashed into his office and saw the lady looking so angry and disappointed.i had no idea what happened there so i asked her. she stammered when she tried to speak. it was apparent that she was agitated.we calmed her down. then she opened up your hr is harassing me. he asked too many uncomfortable questions like when was the last time i had sex. i was shocked. i have never witnessed such a thing in my entire life. what has come over you have you forgotten this is an office my partner was mad at him as well.we apologized to the young lady and fired our hr im